# Data mining untuk Thermal dan SO2 dari web http://mounts-project.com
---

## Import Modules python yang dibutuhkan
Adapun module yang digunakan antara lain:
1. Module `requests` digunakan untuk membuka web http://mounts-project.com
2. Module `json` digunakan untuk melakukan formatting data JSON
3. Module `re` digunakan untuk melakukan ekstraksi data JSON dari variable JavaScript menggunakan Regex
4. Module `pandas` digunakan untuk melakukan formatting data ke DataFrame dan converting ke Excel atau CSV
5. Module `os` digunakan untuk mendapatkan directory dan file
6. Module `JSON` digunakan untuk display data JSON

In [ ]:
import requests
import json
import re
import pandas as pd
import os
from IPython.display import JSON
from datetime import datetime

## Inisiasi Variable yang Dibutuhkan
Inisiasi variable untuk web http://mounts-project.com

In [ ]:
mounts_url = 'http://mounts-project.com/timeseries/'

Inisiasi variable untuk `output_directory` tempat hasil ekstrak data disimpan

In [ ]:
output_directory = os.path.join(os.getcwd(), 'output')

if (not os.path.exists(output_directory)):
    os.mkdir(output_directory)

Variable `volcano_codes` merupakan variable kode gunung api berdasarkan ID smithsonian number (https://volcano.si.edu/).  
Untuk mendapatkan data beberapa gunung api, masukkan kode gunung api dalam bentuk `list`

In [ ]:
volcano_codes = [
    262000, #Anak Krakatau
    261170, #Kerinci
    267020, #Karangetang
    268010, #Dukono
    264230, #Ili Lewotolok
    268030, #Ibu
    263300, #Semeru
    263340, #Raung
    263350, #Ijen
]

Menginisiasi list kosong untuk menyimpan DataFrame hasil ekstraksi data. List ini akan berisi kumpulan data Thermal dan SO2 dari berbagai gunung api

In [ ]:
dataframes = {}

---
## Inisiasi Fungsi-fungsi yang digunakan untuk extract, transformasi, dan export data
Fungsi yang digunakan untuk mengeskstrak variable JSON Thermal dan SO2 dari JavaScript web Mounts Project. Variabel ini tersimpan dengan nama `var_graph`

In [ ]:
def get_json_from_javascript(response):
    var_graph = re.search(r"(?:^|\s|;)var\s+graph\s*=\s*([^']+})", response.text)
    string_graph = var_graph.group(1)
    json_graph = json.loads(string_graph)
    return json_graph

Hasil dari ekstraksi JSON pada fungsi `get_json_from_javascript` selanjutnya digunakan untuk extraksi nilai **SO2** dan **Thermal**

In [ ]:
def get_so2_values(graph_json):
    so2_values = {
        'Date time': graph_json['data'][2]['x'],
        'Value': graph_json['data'][2]['y'],
    }
    so2_df = pd.DataFrame.from_dict(so2_values)
    so2_df['Type'] = 'SO2'
    return so2_df

In [ ]:
def get_thermal_values(graph_json):
    thermal_values = {
        'Date time': graph_json['data'][0]['x'],
        'Value': graph_json['data'][0]['y'],
    }
    thermal_df = pd.DataFrame.from_dict(thermal_values)
    thermal_df['Type'] = 'Thermal'
    return thermal_df

Splitting kolom `Date Time` ke `Date` dan `Time`

In [ ]:
def convert_to_date(date_time):
    return date_time.strftime("%Y-%m-%d")

In [ ]:
def convert_to_time(date_time):
    return date_time.strftime("%H:%M:%S")

Fungsi ini digunakan untuk meng-export data hasil ekstraksi ke dalam format Excel

In [ ]:
def export_to_excel(filtered_df, code):
    filename = '{}.xlsx'.format(code)
    filtered_df.to_excel(os.path.join(output_directory, filename), sheet_name='Join Data')

---
## Kode Utama
Kode utama ekstraksi data

In [ ]:
for code in volcano_codes:
    url = mounts_url+str(code)
    response = requests.get(url)
    graph_json = get_json_from_javascript(response)
    
    df = pd.concat([
        get_so2_values(graph_json),
        get_thermal_values(graph_json)
    ])
    
    df['Date time'] = pd.to_datetime(df['Date time'])
    df['Date'] = df['Date time'].apply(convert_to_date)
    df['Time'] = df['Date time'].apply(convert_to_time)
    df.set_index('Date time', inplace=True)
    
    filtered_df = df[df['Value'] > 0.1]
    dataframes[code] = filtered_df
    export_to_excel(filtered_df, code)
    print('Data untuk gunung api {} berhasil diekstrak, dan disimpan dalam variable dataframes[{}]'.format(code,code))